# Imports

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Funções

## Gerar Metadados

In [ ]:
#Verificar os meta dados do DataFrame
def generate_metadata(dataframe):
    """
    Gera um dataframe contendo metadados das colunas do dataframe fornecido.

    :param dataframe: DataFrame para o qual os metadados serão gerados.
    :return: DataFrame contendo metadados.
    """

    # Coleta de metadados básicos
    metadata = pd.DataFrame({
        'nome_variavel': dataframe.columns,
        'tipo': dataframe.dtypes,
        'qt_nulos': dataframe.isnull().sum(),
        'percent_nulos': round((dataframe.isnull().sum() / len(dataframe))* 100,2),
        'cardinalidade': dataframe.nunique(),
    })
    metadata=metadata.sort_values(by='tipo')
    metadata = metadata.reset_index(drop=True)

    return metadata

# Testando a função com o dataframe dos medicamentos industrializados de novembro de 2021
metadata_df = generate_metadata(df_industrial_medics)

## Boxplots

In [ ]:
def pod_academy_box_plot(dataframe):
    """
    Plota boxplots estilizados com título simplificado para todas as variáveis numéricas do dataframe fornecido
    em um painel com 3 gráficos por linha.

    :param dataframe: DataFrame para o qual os boxplots serão gerados.
    """
    # Seleciona apenas colunas numéricas
    numeric_columns = dataframe.select_dtypes(include=['float64', 'int64']).columns

    # Define o número de linhas com base no número de colunas numéricas
    nrows = len(numeric_columns) // 3 + (len(numeric_columns) % 3 > 0)

    # Inicializa o painel de gráficos
    fig, axes = plt.subplots(nrows=nrows, ncols=3, figsize=(15, nrows * 5))

    # Ajusta o layout
    plt.tight_layout(pad=5)

    # Configura estilo e paleta de cores
    sns.set_style("whitegrid")

    # Plota boxplots para cada coluna numérica
    for i, column in enumerate(numeric_columns):
        sns.boxplot(data=dataframe[column], ax=axes[i//3, i%3], color="skyblue")
        axes[i//3, i%3].set_title(f'{column}', fontdict={'fontsize': 14, 'fontweight': 'bold'})
        axes[i//3, i%3].set_ylabel('')
        axes[i//3, i%3].tick_params(axis='both', which='major', labelsize=12)

    # Remove gráficos vazios (se houver)
    for j in range(i+1, nrows*3):
        fig.delaxes(axes.flatten()[j])

    # Adiciona título principal
    fig.suptitle("Análise descritiva - BoxPlot", fontsize=20, fontweight='bold', y=1.05)

## Histogramas

In [ ]:
def pod_academy_histograms(dataframe):
    """
    Plota histogramas corrigidos com a curva KDE (Kernel Density Estimation) para todas as variáveis numéricas
    do dataframe fornecido em um painel com 3 gráficos por linha.

    :param dataframe: DataFrame para o qual os histogramas serão gerados.
    """
    # Seleciona apenas colunas numéricas
    numeric_columns = dataframe.select_dtypes(include=['float64', 'int64']).columns

    # Define o número de linhas com base no número de colunas numéricas
    nrows = len(numeric_columns) // 3 + (len(numeric_columns) % 3 > 0)

    # Inicializa o painel de gráficos
    fig, axes = plt.subplots(nrows=nrows, ncols=3, figsize=(15, nrows * 5))

    # Ajusta o layout
    plt.tight_layout(pad=4)

    # Configura estilo e paleta de cores
    sns.set_style("whitegrid")

    # Plota histogramas com KDE para cada coluna numérica
    for i, column in enumerate(numeric_columns):
        sns.histplot(data=dataframe[column], ax=axes[i//3, i%3], color="skyblue", bins=30, kde=True)
        axes[i//3, i%3].set_title(f'{column}', fontdict={'fontsize': 14, 'fontweight': 'bold'})
        axes[i//3, i%3].set_ylabel('Frequência')
        axes[i//3, i%3].tick_params(axis='both', which='major', labelsize=12)

    # Remove gráficos vazios (se houver)
    for j in range(i+1, nrows*3):
        fig.delaxes(axes.flatten()[j])

    # Adiciona título principal
    fig.suptitle("Análise descritiva - Histograma com KDE", fontsize=20, fontweight='bold', y=1.05)

## Frequencia categórica

In [ ]:
def plot_categorical_frequency_pt(df, corte_cardinalidade=30, graficos_por_linha=2):
    """
    Plota a frequência de categorias para variáveis categóricas em um DataFrame.

    Parâmetros:
    - df: DataFrame para plotagem.
    - corte_cardinalidade: Cardinalidade máxima para uma coluna ser considerada (padrão é 30).
    - graficos_por_linha: Quantidade de gráficos por linha (padrão é 3).

    Retorna:
    - Exibe os gráficos de barras.
    """

    # Gera metadados para o DataFrame
    metadados = []
    for coluna in df.columns:
        metadados.append({
            'Variável': coluna,
            'Tipo': df[coluna].dtype,
            'Cardinalidade': df[coluna].nunique()
        })

    df_metadados = pd.DataFrame(metadados)

    # Filtra colunas com cardinalidade maior que o corte e tipos não numéricos
    variaveis_categoricas = df_metadados[(df_metadados['Cardinalidade'] <= corte_cardinalidade) & (df_metadados['Tipo'] == 'object')]

    # Calcula o número de linhas e colunas para os subplots
    n_linhas = -(-len(variaveis_categoricas) // graficos_por_linha)  # Ceiling division
    n_colunas = min(len(variaveis_categoricas), graficos_por_linha)

    # Plota as variáveis categóricas
    fig, axs = plt.subplots(nrows=n_linhas, ncols=n_colunas, figsize=(15, 5 * n_linhas))

    for i, (idx, linha) in enumerate(variaveis_categoricas.iterrows()):
        var = linha['Variável']
        ax = axs[i // graficos_por_linha, i % graficos_por_linha]
        df[var].value_counts().sort_index().plot(kind='bar', ax=ax, color='skyblue')
        ax.set_title(f'Frequência em {var}')
        ax.set_ylabel('Frequência')
        ax.set_xlabel(var)

    # Remove os eixos vazios, se houver
    for j in range(i + 1, n_linhas * n_colunas):
        axs[j // graficos_por_linha, j % graficos_por_linha].axis('off')

    plt.tight_layout()
    plt.show()

# Testa a função com os dados e o corte padrão de cardinalidade
plot_categorical_frequency_pt(df, corte_cardinalidade=30, graficos_por_linha=2)


# Objetivo:

Compreender as tendências, padrões e características das vendas farmacêuticas em diferentes regiões e demografias.

In [3]:
# Abrir o CSV dos Medicamentos Industrializados de de Novembro de 2021
df_industrial_medics = pd.read_csv('EDA_Industrializados_202111_sample.csv')

df_industrial_medics.shape

(200000, 16)

# Atividades a serem desenvolvidas

## 1. Limpeza e Pré-processamento de Dados:

* Lidar com valores ausentes: Decidir se vai imputar, preencher ou remover linhas/colunas com dados faltantes.

* Tratar outliers: Identificar e lidar com outliers que podem distorcer a análise.


* Converter tipos de dados, se necessário, garantindo que colunas relacionadas a datas estejam no formato correto.

## 2. Análise Exploratória de Dados (EDA):

* Tendências de Vendas:
    - Vendas mensais: Identificar se há meses com vendas excepcionalmente altas ou baixas.
    - Análise por estado (UF_VENDA): Quais estados têm o maior número de vendas? Existem padrões regionais?

* Produtos Mais Vendidos:
    - Baseado no PRINCIPIO_ATIVO: Quais são os medicamentos mais populares?
    - Baseado na DESCRICAO_APRESENTACAO: Quais apresentações ou dosagens são mais vendidas?


* Demografia dos Consumidores:
    - Distribuição de idade dos compradores.
    - Distribuição de gênero dos compradores.


* Correlações:
    - Existe uma correlação entre o tipo de medicamento e a idade do comprador?
    - Existe uma correlação entre o gênero do comprador e o tipo de medicamento?


## 3. Visualização de Dados:

* Gráficos de barras para mostrar as vendas por estado, por medicamento e por gênero.


* Histogramas para distribuição de idade.


* Heatmaps para correlações.


* Gráficos de linha para tendências de vendas ao longo dos meses.

## 4. Insights e Recomendações:

* Com base na EDA, identifique insights interessantes sobre os hábitos de compra e padrões.


* Formule recomendações para farmácias ou empresas farmacêuticas com base nas tendências observadas.

# Montar uma apresentação com até 5 slides sobre minha análise

### Slide 1: Título e Introdução

* Título: "Análise de Vendas Farmacêuticas"
* Subtítulo: Uma visão geral das vendas de medicamentos.

### Slide 2: Metodologia e Limpeza de Dados


* Título: "Metodologia e Preparação de Dados"
* Pontos Principais:
    - Breve descrição dos dados: O que cada coluna representa.
    - Como os valores ausentes foram tratados.
    - Como os outliers foram identificados e tratados.
* Gráfico/Imagem: Uma visualização que mostre, por exemplo, a quantidade de dados ausentes por coluna ou um gráfico de caixa mostrando a distribuição de uma variável-chave.

### Slide 3: Tendências e Padrões de Vendas



* Título: "Tendências de Vendas por Região e Tempo"
* Pontos Principais:
    - Vendas totais por estado.
    - Meses com maiores e menores vendas.
* Gráficos:
    - Gráfico de barras mostrando vendas por estado.
    - Gráfico de linha mostrando a tendência de vendas ao longo dos meses.

### Slide 4: Informações Demográficas


* Título: "Perfil do Consumidor"
* Pontos Principais:
    - Distribuição de idade dos compradores.
    - Distribuição de gênero dos compradores.
* Gráficos:
    - Histograma para distribuição de idade.
    - Gráfico de barras ou torta para distribuição de gênero.

### Slide 5: Conclusões e Recomendações


* Título: "Insights e Próximos Passos"
* Pontos Principais:
    - Quais medicamentos ou apresentações são os mais vendidos e onde.
    - Insights sobre preferências demográficas.
    - Recomendações para estratégias de marketing ou distribuição.
* Imagem: Uma imagem que represente estratégia ou planejamento.